In [1]:
!git clone https://github.com/peppermenta/faceHack

fatal: destination path 'faceHack' already exists and is not an empty directory.


In [2]:
!pip3 install -r faceHack/requirements.txt

In [3]:
import torch
from PIL import Image
import numpy as np
import os      
import torchvision         

class FaceDataset(torch.utils.data.Dataset):
  def __init__(self,train=True,type='normal',transform=None):
    super().__init__()
    if(train):
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','train.csv'),delimiter=',',dtype=None,encoding='utf-8')
    else:
      self.data = np.genfromtxt(os.path.join('faceHack','dataset','test.csv'),delimiter=',',dtype=None,encoding='utf-8')

    self.type=type
    self.transform = transform

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index):
    img_name = self.data[index][0]
    X = Image.open(os.path.join('faceHack','dataset','happy_images','{}.jpg'.format(img_name)))
    label = self.data[index][1]

    if(self.type=='normal'):
      if(label=='NOT smile'):
        y = 0
      else:
        y = 1
    else:
      if(label=='positive smile'):
        y = 1
      else:
        y = 0

    if(self.transform):
      X = self.transform(X)
    return X,y


In [4]:
faceTransform = torchvision.transforms.Compose([
  torchvision.transforms.Resize(224),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.ToTensor(),
])

In [8]:
device = torch.device('cuda')
print(device)
# model = torch.hub.load('pytorch/vision:v0.9.0', 'resnext101_32x8d', pretrained=True)
model = torchvision.models.resnet18(pretrained=True)

# for param in model.parameters():
#   param.requires_grad = False

in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features=in_features,out_features=2)
model = model.to(device)


trainDataset = FaceDataset(transform=faceTransform)
testDataset = FaceDataset(train=False,transform=faceTransform)
trainLoader = torch.utils.data.DataLoader(trainDataset,batch_size=32,shuffle=True,num_workers=2)
testLoader = torch.utils.data.DataLoader(testDataset,batch_size=32,shuffle=False,num_workers=2)

cuda


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [11]:
model.train()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.00001)
epochs = 20

loss_hist = []

for epoch in range(epochs):
  tot_loss = 0
  correct = 0
  for i,data in enumerate(trainLoader):
    X,y = data
    X = X.to(device)
    y = y.to(device)

    out = model(X)

    _,pred = torch.max(out,dim=1)
    correct += (pred==y).sum().item()

    loss = criterion(out,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    tot_loss += loss.item()
    if(i%20==0):
      print('Epoch:({},{}): Loss={}'.format(epoch,i,loss.item()))

  loss_hist.append(tot_loss)
  print('Accuracy is {}'.format(correct/len(trainDataset)))


import matplotlib.pyplot as plt

plt.plot([i for i in range(epochs)],loss_hist)


Epoch:(0,0): Loss=0.21024098992347717
Epoch:(0,20): Loss=0.24143952131271362
Epoch:(0,40): Loss=0.1771562695503235
Epoch:(0,60): Loss=0.09179981797933578
Epoch:(0,80): Loss=0.1920701563358307
Epoch:(0,100): Loss=0.18343539535999298
Epoch:(0,120): Loss=0.09841056913137436
Epoch:(0,140): Loss=0.32677021622657776
Accuracy is 0.9223763196025667
Epoch:(1,0): Loss=0.21512922644615173
Epoch:(1,20): Loss=0.10100206732749939
Epoch:(1,40): Loss=0.4533546268939972
Epoch:(1,60): Loss=0.16296620666980743
Epoch:(1,80): Loss=0.14897404611110687
Epoch:(1,100): Loss=0.18975397944450378
Epoch:(1,120): Loss=0.18384794890880585
Epoch:(1,140): Loss=0.2706131041049957
Accuracy is 0.945352929000207
Epoch:(2,0): Loss=0.1782318353652954
Epoch:(2,20): Loss=0.11088445037603378
Epoch:(2,40): Loss=0.09868742525577545
Epoch:(2,60): Loss=0.1418096274137497
Epoch:(2,80): Loss=0.0834188312292099
Epoch:(2,100): Loss=0.056571222841739655
Epoch:(2,120): Loss=0.19595129787921906
Epoch:(2,140): Loss=0.08416631817817688
Acc

KeyboardInterrupt: ignored

In [ ]:
model.eval()